# Llama3



In [77]:
import sys
import json
import re
import uuid
from io import BytesIO
import base64

import gradio as gr
import matplotlib.pyplot as plt
# Chat completions API
import openai
from google.auth import default, transport
from google.cloud import storage
from PIL import Image
PROJECT_ID = "mimetic-kit-445917-d8" 
# Only `us-central1` is supported region for Llama 3.2 models using Model-as-a-Service (MaaS).
LOCATION = "us-central1"
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)
credentials, _ = default()
auth_request = transport.requests.Request()
credentials.refresh(auth_request)

MODEL_LOCATION = "us-central1"
MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

client = openai.OpenAI(
    base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
    api_key=credentials.token,
)

MODEL_ID = "meta/llama-3.2-90b-vision-instruct-maas"  # @param {type:"string"} ["meta/llama-3.2-90b-vision-instruct-maas"]

In [91]:
import json
import re
import uuid
from io import BytesIO

import gradio as gr
import matplotlib.pyplot as plt
# Chat completions API
import openai
from google.auth import default, transport
from google.cloud import storage
from PIL import Image

def visualize_image_from_bucket(bucket_name: str, blob_name: str) -> None:
    """Visualizes an image stored in a Google Cloud Storage bucket."""
    try:
        # Create a client for interacting with Google Cloud Storage
        storage_client = storage.Client()

        # Get a reference to the bucket and blob
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)

        # Download the image data into memory
        image_data = blob.download_as_bytes()

        # Open the image using PIL
        image = Image.open(BytesIO(image_data))

        # Display the image using matplotlib
        plt.figure(figsize=(10, 10))  # Set the figure size (adjust as needed)
        plt.imshow(image)
        plt.axis("off")  # Turn off axis labels
        plt.show()

    except Exception as e:
        print(f"Error visualizing image: {e}")
        
def encode_image_to_base64(file_path):
    with open(file_path, "rb") as image_file:
        image_data = image_file.read()
        base64_image = base64.b64encode(image_data).decode('utf-8')
        return f"data:image/jpeg;base64,{base64_image}"
      
def create_collage(image_paths, output_path):
    images = [Image.open(img) for img in image_paths]
    width, height = images[0].size

    # Combine images horizontally
    collage = Image.new("RGB", (width * len(images), height))
    for i, img in enumerate(images):
        collage.paste(img, (i * width, 0))

    collage.save(output_path)
      



In [95]:
prompt = """
"You are a causal discovery expert. Your task is to analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions.There are 3 variables: 
1. neddle_position
2. magnetic_bar_direction
3. neddle_direction
.
Please fill this causality adjacency matrix:
[[_, _, _]
 [_, _, _]
 [_, _, _]].
In the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
"""

image_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_magnetic_128P/20241216_004627_Over_3D_128p.png"
# Getting the base64 string


image_paths = [image_path]
output_path = "./collage.png"
create_collage(image_paths, output_path)
with open(output_path, "rb") as image_file:
    base64_collage = base64.b64encode(image_file.read()).decode("utf-8")

base64_image = encode_image(image_path)

prompt = "what do you see in the images"



response = client.chat.completions.create(
    model=MODEL_ID,
    messages=[
        {
            "role": "user",
            "content": [
                {"image_url": {"url": "data:image/png;base64," + base64_collage}, "type": "image_url"},
                {"text": prompt, "type": "text"},
            ],
        },
    ],
    max_tokens=max_tokens,
)

print(response.choices[0].message.content)

The image shows a 3D bar graph with a red and blue color scheme. The graph features a single bar that is divided into two sections, with the top section being blue and the bottom section being red. The bar is positioned at an angle, with the top section pointing towards the upper left corner of the image and the bottom section pointing towards the lower right corner.

To the left of the bar, there is a small red and blue triangle that appears to be a legend or key for the graph. The triangle is oriented in the same direction as the bar, with the red section at the bottom and the blue section at the top.

The background of the image is white, which helps to make the colors of the bar and triangle stand out. Overall, the image presents a clear and concise visual representation of data, with the use of color and orientation helping to convey meaning and context.
